In [1]:
import numpy as np

In [2]:
# Enter the number of layers of your network
num_layers = int(input("Enter the number of layers : "))

# Enter the number of neurons of your network
split_int = lambda x : int(x.split(' '))
num_neurons = input("Enter the number of neurons of each layer in your network")

split_int(num_neurons)

5


In [14]:
# Construct a list that contains the number of neurons of each layer in your network.
structure = input("Enter the number of neurons of each layer in your network").split(' ')
structure = [int(x) for x in structure]
print(structure)    

[2, 5, 3]
